## 说明
之前所有输入都在一个txt文件中，一行代表一个报文，忽略了报文中的回车换行符，并且默认了报文中不含有符号'-'  
本处做修正：  
1. 每个txt文件作为一个报文
2. 不能采用-作为分隔符，选用####作为连接各个n-gram的分隔符
3. 由于http自己构造时有大量随机字符，导致重复的n-gram很少，采用先统计unique_ngrams的方式反而会使时间加倍，因此不能先统计unique_ngrams，而是直接使用all_ngrams

In [1]:
import os
import time

In [2]:
# 定义函数，读取所有的输入txt文件，获得所有的报文数据
# 参数：文件夹名称，txt后缀
# 返回值：所有报文构成的列表，列表每个元素代表一条报文
def read_input(packet_name):
    
    packets = []  # 存放所有报文，每个元素代表一个报文
    file_list = os.listdir(packet_name)
    num = len(file_list)  # 文件夹下报文txt个数
    
    for i in range(1, num+1):
        
        # 每个文件名
        read_file_name = packet_name + '\\' + str(i) + '.txt'
    
        # 打开，二进制整体读取，而非按行读取
        f = open(read_file_name, 'r')
        content = f.read()
        f.close()
        
        # 存入列表中
        packets.append(content)
    
    return packets

In [3]:
# 定义函数，由所有报文得到不同的n-grams，列表中不会出现相同的元素。目的是提升统计频率的时间效率
# 参数：第一个参数为n，便于尝试不同的n值；第二个参数为packets，即待划分的报文数据列表
# 返回值：所有不同的 n-grams 构成的列表
def get_unique_ngrams(n, packets):
    num = len(packets)  # 报文数目
    unique_ngrams = []
    for i in range(num):  # 遍历所有报文
        message = packets[i]
        length = len(message)  # 长度
        repeat = length - n + 1  # 一条报文上的循环次数
        for i in range(repeat):
            if message[i:i+n] not in unique_ngrams:
                unique_ngrams.append(message[i:i+n])  # 都存入ngrams列表中
    return unique_ngrams

In [4]:
# 定义函数，由所有报文得到n-grams，此处还未做频率的统计处理
# 参数：第一个参数为n，便于尝试不同的n值；第二个参数为packets，即待划分的报文数据列表
# 返回值：所有 n-grams 构成的列表
def get_all_ngrams(n, packets):
    num = len(packets)  # 报文数目
    all_ngrams = []
    for i in range(num):  # 遍历所有报文
        message = packets[i]
        length = len(message)  # 长度
        repeat = length - n + 1  # 一条报文上的循环次数
        for i in range(repeat):
            all_ngrams.append(message[i:i+n])  # 都存入ngrams列表中
    return all_ngrams

In [5]:
# 定义函数，只保留频率排名靠前的一部分n-grams
# 参数：第一个参数为 P ，代表保留频率和满足阈值的一部分n-grams；
#       第二个参数为unique_ngrams，即所有不同的n-grams的列表(get_unique_ngrams的返回值)
#       第三个参数为all_ngrams，即所有报文的n-grams的列表(get_all_ngrams的返回值)
# 返回值：频率和为P的 n-grams 列表
def get_allfreq_ngrams(P, all_ngrams):
    # 统计n-grams总数
    num = len(all_ngrams)
    print("所有报文的n-grams总数为：", num)
    # 先统计每种n-gram的出现频率，记录在字典中
    ngrams_dict = {}  # 存放各个n-gram的频率信息，无序的
    for i in all_ngrams:  # 遍历n-grams，统计每个出现频率
        ngrams_dict[i] = all_ngrams.count(i)/num  # key为n-gram, value为出现的频率
    # 对频率进行由大到小排序
    ordered = sorted(ngrams_dict.items(), key=lambda item: item[1], reverse=True)
    # 取出频率较高的一部分一部分
    allfreq_ngrams = []
    i = 0  # 当前取到第几个
    sum_p = 0  # 频率和
    while(sum_p < P):
        sum_p += ordered[i][1]
        allfreq_ngrams.append(ordered[i][0])
        i += 1
    return allfreq_ngrams

In [6]:
# 定义函数，由所有报文得到n-grams，此处还未做频率的统计处理
# 参数：第一个参数为n，便于尝试不同的n值；第二个参数为packets，即待划分的报文数据列表
# 返回值：列表套列表，大列表中每个元素是一个小列表，小列表对应一条报文的n-grams
def get_separate_ngrams(n, packets):
    sep_ngrams = []  # 用于存放不同报文的n-grams列表的大列表
    num = len(packets)
    for i in range(num):
        message = packets[i]
        length = len(message)
        message_ngrams = []  # 存储该报文n-grams的小列表
        repeat = length - n + 1  # 循环次数
        for j in range(repeat):
            message_ngrams.append(message[j:j+n])  # 小列表
        sep_ngrams.append(message_ngrams)  # 大列表
    return sep_ngrams

In [7]:
# 定义函数，每个报文只保留频率较高的n-grams
# 参数：第一个参数为 freq_ngrams，频数排名靠前的n-grams列表，即函数get_freq_ngrams的返回结果
#       第二个参数为 ngrams, 各个报文n-grams列表组成的列表，即函数get_separate_ngrams的返回结果，大列表套小列表形式
# 返回值：列表套列表，大列表中每个元素是一个小列表，小列表对应一条报文的高频n-grams
def get_sepfreq_ngrams(allfreq_ngrams, ngrams):
    sepfreq_ngrams = []  # 用于存放不同报文的高频n-grams列表的大列表
    num = len(ngrams)
    for i in range(num):  # 遍历报文
        message = ngrams[i]
        message_freq_ngrams = []  # 该报文高频n-grams的小列表
        for j in range(len(message)):
            if message[j] in allfreq_ngrams:
                message_freq_ngrams.append(message[j])  # 小列表
        sepfreq_ngrams.append(message_freq_ngrams)  # 大列表
    return sepfreq_ngrams

In [8]:
# 定义函数，将每个报文的高频n-grams保存到文件中,同一报文中n-grams空格隔开，每个报文占一行，末尾加换行符
# 参数：写入的内容，即报文的高频n-grams列表组成的大列表
def save_freq_file(packet_name, content):
    
    for i in range(1, len(content)+1):
        # 写入文件名
        write_file_name = packet_name + '\\' + str(i) + '.txt'
        f = open(write_file_name, 'w')
        
        s = content[i-1]  # 减1，列表从0开始编号
        t = '####'.join(s)
        f.write(t)
        
        f.close()  # 关闭文件

In [17]:
begin_time = time.time()  # 统计程序运行时间

n = 4  # 初始化取值，n-gram中 n 的取值
P = 0.2  # 保留的频率和

packets = read_input('input')  # 读取文件，得到报文列表
print("成功读取输入文件")

all_ngrams = get_all_ngrams(n, packets)  # 从输入文件中获得所有n-grams的列表
print("成功获得所有n-grams的小列表")

allfreq_ngrams = get_allfreq_ngrams(P, all_ngrams)  # 排序，取出频率较高的一部分n-grams
ngrams_num = len(allfreq_ngrams)
print("\n保留频率和为%f的n-grams，共保留%d个" % (P, ngrams_num))
print("保留的n-grams为：\n", allfreq_ngrams)

sep_ngrams = get_separate_ngrams(n, packets)  # 从输入文件中获得n-grams的大列表套小列表的形式
print("\n成功获得各个报文的n-grams列表")

sepfreq_ngrams = get_sepfreq_ngrams(allfreq_ngrams, sep_ngrams)  # 保留每条报文中的高频n-grams
print("成功获得各个报文的高频n-grams列表")

save_freq_file('freq-ngrams', sepfreq_ngrams)  # 保存报文的高频n-grams
print("成功保存各个报文的高频n-grams到文件中")

end_time = time.time()
run_time = end_time - begin_time
print ('\n该模块运行时间：',run_time, '秒')

成功读取输入文件
成功获得所有n-grams的小列表
所有报文的n-grams总数为： 23948

保留频率和为0.200000的n-grams，共保留18个
保留的n-grams为：
 ['1900', 'ee30', '0236', '2361', '3619', '6190', '08ee', '8ee3', 'e307', '307b', '9002', '0027', '9003', '003c', '9001', '001d', '9008', '0081']

成功获得各个报文的n-grams列表
成功获得各个报文的高频n-grams列表
成功保存各个报文的高频n-grams到文件中

该模块运行时间： 10.665493726730347 秒
